<a href="https://colab.research.google.com/github/bhashanasirimanna/forecasting_test/blob/main/weatherForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Section 01: import Libraries**

In [53]:
import requests #this libraray helps us to fetch data from API
import pandas as pd # for handling and analysing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #for splitting data into train and test
from sklearn.preprocessing import LabelEncoder #to convert catogerical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regressin
from sklearn.metrics import mean_squared_error #for evaluating models
from datetime import datetime, timedelta #for handling date and time
import pytz

In [54]:
API_KEY = 'c1406b25c6c34edb8974a6a332d4d0b9'
BASE_URL = 'https://api.openweathermap.org/data/2.5/' #base url for making API requests

**1. fetch current weather Data**

In [55]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #construct the API request URL
    response = requests.get(url) #send the API request and get the response
    data = response.json()
    return{
        'city': data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like' : round(data['main']['feels_like']),
        'temp_min' : round(data['main']['temp_min']),
        'temp_max' : round(data['main']['temp_max']),
        'humidity' : round(data['main']['humidity']),
        'description' : data['weather'][0]['description'],
        'country' : data['sys']['country'],
        'wind_gust_dir' : data['wind']['deg'],
        'pressure' : data['main']['pressure'],
        'wind_gust_speed' : data['wind']['speed'],
    }

**2. Read Historical Data**

In [56]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

**3. Prepare data for training**

In [57]:
def prepare_data(data):
  le = LabelEncoder() #create a labelEncoder instance
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define the feature variable and target variables
  x = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] #feature variables
  y= data['RainTomorrow'] #target variable

  return x, y, le #return feature variable, traget variable and the label encoder

**4. Train Rain Prediction Model**

In [58]:
def train_rain_model(x, y):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) #split the data into train and test sets
  model = RandomForestClassifier(n_estimators=100, random_state=42) #create a random forest
  model.fit(x_train, y_train) #train the model

  y_pred = model.predict(x_test) #make predictions on the test set

  # print('Accuracy:', accuracy_score(y_test, y_pred)) #evaluate the model
  print ('Mean squared Error For Rain Model')

  print(mean_squared_error(y_test, y_pred))

  return model

**5. Prepare regressinon data**

In [59]:
def prepare_regression_data(data, feature):
  x, y = [], [] #initialize list for feature and target values

  for i in range(len(data) - 1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i + 1])

  x = np.array(x).reshape(-1, 1)
  y = np.array(y)

  return x, y

**Train regression model**

In [60]:
def train_regression_model(x, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x, y)

  return model


**Predict future**

In [61]:
def predict_future(model, current_value):
  predictions = [current_value]
  for _ in range(7):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]


**Weather analysis function**

In [62]:
from multiprocessing import current_process
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)
  print(current_weather)

  #load historical data
  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train tha rain prediction model
  x, y, le = prepare_data(historical_data)

  rain_model = train_rain_model(x, y)

  #map wind direction to campass points
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ('N', 0, 11.25), ('NNE', 11.25, 33.75), ('NE', 33.75, 56.25),
      ('ENE', 56.25, 78.75), ('E', 78.75, 101.25), ('ESE', 101.25, 123.75),
      ('SE', 123.75, 146.25), ('SSE', 146.25, 168.75), ('S', 168.75, 191.25),
      ('SSW', 191.25, 213.75), ('SW', 213.75, 236.25), ('WSW', 236.25, 258.75),
      ('W', 258.75, 281.25), ('WNW', 281.25, 303.75), ('NW', 303.75, 326.25),
      ('NNW', 326.25, 348.75),
      ('N', 348.75, 360) # Added to handle wrap-around for North
  ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_gust_speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }

  current_df = pd.DataFrame([current_data])

  #rain prediction

  rain_prediction = rain_model.predict(current_df)[0]

  #prepare regression model for temperature and humidity

  temp_x, temp_y = prepare_regression_data(historical_data, 'Temp')

  x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(temp_x, temp_y)

  hum_model = train_regression_model(x_hum, y_hum)

  #predict future temperature and humidity

  future_temp = predict_future(temp_model, current_weather['temp_min'])

  future_humidity = predict_future(hum_model, current_weather['humidity'])

  #prepare time for future predictions

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #Display results

  print(f"City: {city}, {current_weather['current_temp']} ")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather prediction: {current_weather['description']}")
  print(f"Rain prediction: {'Yes' if rain_prediction == 1 else 'No'}")


  print(f"\nFuture Temperature Predictions:")
  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print(f"\nFuture Humidity Predictions:")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()

Enter any city name: puttalam
{'city': 'Puttalam', 'current_temp': 28, 'feels_like': 31, 'temp_min': 28, 'temp_max': 28, 'humidity': 75, 'description': 'scattered clouds', 'country': 'LK', 'wind_gust_dir': 355, 'pressure': 1011, 'wind_gust_speed': 6.8}
Mean squared Error For Rain Model
0.1506849315068493
City: puttalam, 28 
Current Temperature: 28°C
Feels Like: 31°C
Minimum Temperature: 28°C
Maximum Temperature: 28°C
Humidity: 75%
Weather prediction: scattered clouds
Rain prediction: Yes

Future Temperature Predictions:
11:00: 27.7°C
12:00: 22.8°C
13:00: 25.4°C
14:00: 24.7°C
15:00: 23.4°C

Future Humidity Predictions:
11:00: 46.1%
12:00: 48.2%
13:00: 53.7%
14:00: 58.1%
15:00: 54.2%
